In [1]:
###Dependecies
import sqlite3
import pandas as pd
from io import BytesIO
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time
import numpy as np
from config import skey
import json
import pprint as pp
from collections import OrderedDict 
from get_data import demystify, basic_user_data, advanced_user_data


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\pwitw\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [13]:
pd.options.display.float_format = '{:.2f}'.format
# Retrieve page with the requests module
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\pwitw\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [3]:
#my steam id
my_id = 76561198068112781

In [19]:
temp_steam_id = 76561198048037824
player_summary_url = f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={skey}&steamids=76561198019605290"
response = requests.get(player_summary_url).json()
#print(steam_ids[0])
#print(response)

In [20]:
print(json.dumps(response, indent=4, sort_keys=True))

{
    "response": {
        "players": [
            {
                "avatar": "https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/e4/e4829f95901bcd959b221e96c977a83735b6a447.jpg",
                "avatarfull": "https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/e4/e4829f95901bcd959b221e96c977a83735b6a447_full.jpg",
                "avatarhash": "e4829f95901bcd959b221e96c977a83735b6a447",
                "avatarmedium": "https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/e4/e4829f95901bcd959b221e96c977a83735b6a447_medium.jpg",
                "commentpermission": 1,
                "communityvisibilitystate": 3,
                "lastlogoff": 1615546631,
                "loccountrycode": "TH",
                "personaname": "Al Capwn",
                "personastate": 0,
                "personastateflags": 0,
                "primaryclanid": "103582791434277245",
                "profilestate": 1,
                "profileurl": "htt

In [21]:
#print(response['response']['players'][0].get("personaname"))
temp_user_dict = OrderedDict()
temp_user_dict.update({"user_steam_id" : temp_steam_id})
temp_user_dict

OrderedDict([('user_steam_id', 76561198048037824)])

In [22]:
temp_user_dict.update({"current_account_name" : response['response']['players'][0].get("personaname")})
temp_user_dict.update({"original_account_name" : response['response']['players'][0].get("realname")})
temp_user_dict.update({"profile_url" : response['response']['players'][0].get("profileurl")})


In [23]:
pp.pprint(temp_user_dict)

OrderedDict([('user_steam_id', 76561198048037824),
             ('current_account_name', 'Al Capwn'),
             ('original_account_name', 'Nawal'),
             ('profile_url', 'https://steamcommunity.com/id/nawaldua/')])


In [24]:
owned_games_url = f"http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={skey}&steamid=76561198019605290&format=json"
game_list = requests.get(owned_games_url).json()
print(json.dumps(game_list, indent=2))

{
  "response": {
    "game_count": 81,
    "games": [
      {
        "appid": 4700,
        "playtime_forever": 0,
        "playtime_windows_forever": 0,
        "playtime_mac_forever": 0,
        "playtime_linux_forever": 0
      },
      {
        "appid": 4000,
        "playtime_forever": 161,
        "playtime_windows_forever": 0,
        "playtime_mac_forever": 0,
        "playtime_linux_forever": 0
      },
      {
        "appid": 17470,
        "playtime_forever": 1044,
        "playtime_windows_forever": 0,
        "playtime_mac_forever": 0,
        "playtime_linux_forever": 0
      },
      {
        "appid": 24740,
        "playtime_forever": 2427,
        "playtime_windows_forever": 0,
        "playtime_mac_forever": 0,
        "playtime_linux_forever": 0
      },
      {
        "appid": 10180,
        "playtime_forever": 1640,
        "playtime_windows_forever": 0,
        "playtime_mac_forever": 0,
        "playtime_linux_forever": 0
      },
      {
        "appid": 1

In [25]:
user_game_stats = {}
play_count = 0

In [26]:

for i in range(game_list["response"]["game_count"]):
    
    #initialize temporary variables
    appid = 0
    hours_played = 0
    
    if game_list["response"]["games"][i].get("playtime_forever") != 0:
        appid = game_list["response"]["games"][i].get("appid")
        hours_played = game_list["response"]["games"][i].get("playtime_forever")
        #function to get appid actual name
        play_count += 1
        
#         dict_name = f"game{play_count}"
         
        user_game_stats.update({ f"game{play_count}" : {"appid" : appid,
                                                        "hours played" : "{:.2f}".format(hours_played/60)
                                                       }
                               })
    #print(user_game_stats)
        
temp_user_dict.update({"total_games_played" : play_count})
temp_user_dict.update({"user_game_data" : user_game_stats})



In [27]:
#pp.pprint(user_game_stats)
pp.pprint(temp_user_dict, indent = 4)

OrderedDict([   ('user_steam_id', 76561198048037824),
                ('current_account_name', 'Al Capwn'),
                ('original_account_name', 'Nawal'),
                ('profile_url', 'https://steamcommunity.com/id/nawaldua/'),
                ('total_games_played', 50),
                (   'user_game_data',
                    {   'game1': {'appid': 4000, 'hours played': '2.68'},
                        'game10': {'appid': 620, 'hours played': '36.13'},
                        'game11': {'appid': 22650, 'hours played': '0.87'},
                        'game12': {'appid': 48000, 'hours played': '4.35'},
                        'game13': {'appid': 42680, 'hours played': '12.70'},
                        'game14': {'appid': 42690, 'hours played': '94.33'},
                        'game15': {'appid': 24240, 'hours played': '4.90'},
                        'game16': {'appid': 205790, 'hours played': '0.03'},
                        'game17': {'appid': 500, 'hours played': '7.48'},


In [14]:
print(demystify(42700))

Call of Duty: Black Ops


In [28]:
owned_games_url = f"http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={skey}&steamid=76561198019605290&format=json"
#76561198072641044
#76561198048037824
#76561198319935141 - game_count: 0
#76561198019605290
game_list = requests.get(owned_games_url).json()

#print(json.dumps(game_list, indent=2))
print(game_list['response']['game_count'])

81


In [16]:
game_ids = {}
steamid = 76561198019605290
#api url for user's game data
owned_games_url = f"http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={skey}&steamid={steamid}&format=json"
game_list = requests.get(owned_games_url).json()

print(f"found {game_list['response']['game_count']} total games for id: {steamid}")
#intialize temporary dictionary container
user_game_stats = {}

play_count = 0
for i in range(game_list["response"]["game_count"]):

    #initialize temporary variables
    appid = 0

    try:
        if game_list["response"]["games"][i].get("playtime_forever") != 0:
            play_count += 1
            #pull game id
            appid = game_list["response"]["games"][i].get("appid")
            identifier = str(appid)
            #name_of_the_game = demystify(identifier)
            #print(name_of_the_game)
            #name_of_the_game = "game"+str(i)
            #identify the game
            if identifier not in game_ids:
                name_of_the_game = demystify(appid)
            elif identifier in game_ids:
                name_of_the_game = game_ids[appid]

            hours_played = game_list["response"]["games"][i].get("playtime_forever")

            user_game_stats.update({name_of_the_game : {"appid" : appid,
                                                        "hours played" : "{:.2f}".format(hours_played/60)}
                                   })
    except:
        "KeyError"

temp_user_dict.update({"total_games_played" : play_count})
temp_user_dict.update({"user_game_data" : user_game_stats})
pp.pprint(temp_user_dict, indent=2)

found 81 total games for id: 76561198019605290
OrderedDict([ ('user_steam_id', 76561198048037824),
              ('current_account_name', 'Al Capwn'),
              ('original_account_name', 'Nawal'),
              ('profile_url', 'https://steamcommunity.com/id/nawaldua/'),
              ('total_games_played', 50),
              ( 'user_game_data',
                { 'Alien Breed 2: Assault': { 'appid': 22650,
                                              'hours played': '0.87'},
                  'Borderlands GOTY': {'appid': 8980, 'hours played': '130.30'},
                  'Burnout Paradise: The Ultimate Box': { 'appid': 24740,
                                                          'hours played': '40.45'},
                  'Call of Duty: Black Ops': { 'appid': 42700,
                                               'hours played': '15.58'},
                  'Call of Duty: Black Ops - Multiplayer': { 'appid': 42710,
                                                             'ho

In [7]:
base_appid_url = "https://steamdb.info/app/"
# Retrieve page with the requests module
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_win32.zip


[WDM] - Driver has been saved in cache [C:\Users\pwitw\.wdm\drivers\chromedriver\win32\89.0.4389.23]


In [33]:
browser.visit(base_appid_url + "10190")
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [34]:
game_title = soup.find('h1').get_text()
print(game_title)

Call of Duty: Modern Warfare 2 - Multiplayer


In [19]:
#test friendslist api call
target_userid = "76561198068112781"
friend_list_url = f"http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={skey}&steamid={target_userid}&relationship=friend"

friends_list_test = requests.get(friend_list_url).json()
print(json.dumps(friends_list_test, indent=2))

{
  "friendslist": {
    "friends": [
      {
        "steamid": "76561197965068085",
        "relationship": "friend",
        "friend_since": 1364581124
      },
      {
        "steamid": "76561197982885742",
        "relationship": "friend",
        "friend_since": 1483498704
      },
      {
        "steamid": "76561197998865837",
        "relationship": "friend",
        "friend_since": 1352849599
      },
      {
        "steamid": "76561198003342479",
        "relationship": "friend",
        "friend_since": 1371532975
      },
      {
        "steamid": "76561198004535097",
        "relationship": "friend",
        "friend_since": 1521160508
      },
      {
        "steamid": "76561198004573642",
        "relationship": "friend",
        "friend_since": 1379718566
      },
      {
        "steamid": "76561198012686814",
        "relationship": "friend",
        "friend_since": 1386993281
      },
      {
        "steamid": "76561198014700313",
        "relationship": "friend"

In [25]:
print(len(friends_list_test["friendslist"]["friends"]))
current_friendslist_length = len(friends_list_test["friendslist"]["friends"])
print(friends_list_test["friendslist"]["friends"][0].get("steamid"))

33
76561197965068085


In [26]:
steamid_collection = []
for i in range(current_friendslist_length):
    steamid_collection.append(friends_list_test["friendslist"]["friends"][i].get("steamid"))

In [27]:
len(steamid_collection) == len(set(steamid_collection))
print(len(steamid_collection))

33


In [24]:
target_userid = 76561198068112781
friend_list_url = f"http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={skey}&steamid={target_userid}&relationship=friend"

friends_list_test = requests.get(friend_list_url).json()
print(json.dumps(friends_list_test, indent=2))

{
  "friendslist": {
    "friends": [
      {
        "steamid": "76561197965068085",
        "relationship": "friend",
        "friend_since": 1364581124
      },
      {
        "steamid": "76561197982885742",
        "relationship": "friend",
        "friend_since": 1483498704
      },
      {
        "steamid": "76561197998865837",
        "relationship": "friend",
        "friend_since": 1352849599
      },
      {
        "steamid": "76561198003342479",
        "relationship": "friend",
        "friend_since": 1371532975
      },
      {
        "steamid": "76561198004535097",
        "relationship": "friend",
        "friend_since": 1521160508
      },
      {
        "steamid": "76561198004573642",
        "relationship": "friend",
        "friend_since": 1379718566
      },
      {
        "steamid": "76561198012686814",
        "relationship": "friend",
        "friend_since": 1386993281
      },
      {
        "steamid": "76561198014700313",
        "relationship": "friend"

In [64]:
print(steamid_collection[5])


76561197966111577


In [146]:
testing = str(steamid_collection[5])
print(testing)

76561197966111577
